In [117]:
import csv
from os import listdir
from os.path import join, abspath
from codecs import open
from lxml import etree
from typing import List, Union, Tuple, Dict

In [39]:
directory = join(abspath('..'), abspath('dataset/Annotations'))

In [100]:
limit = -1 # no limit
annotations = {}
for xmlFile in listdir(directory):
    decrease = 1
    if limit == 0:
        break
    elif limit == -1:
        decrease = 0
    if not xmlFile.endswith('.xml'):
        continue
    trees = etree.parse(directory + '/' + xmlFile)
    annotations[xmlFile[:-4]] = []
    for obj in trees.findall('object'):
        bdb = obj.find('bndbox')
        name = obj.find('name').text
        xmin = bdb.find('xmin').text
        ymin = bdb.find('ymin').text
        xmax = bdb.find('xmax').text
        ymax = bdb.find('ymax').text
        annotations[xmlFile[:-4]].append((f'dataset/JPEGImages/{xmlFile[:-4]}.jpg', xmin, ymin, xmax, ymax, name))
    limit -= decrease

In [101]:
print(annotations['00000']) # annotations[image_name]

[('dataset/JPEGImages/00000.jpg', '9', '111', '61', '182', 'white'), ('dataset/JPEGImages/00000.jpg', '95', '138', '143', '197', 'white'), ('dataset/JPEGImages/00000.jpg', '185', '98', '235', '166', 'white'), ('dataset/JPEGImages/00000.jpg', '262', '125', '311', '192', 'white'), ('dataset/JPEGImages/00000.jpg', '334', '89', '382', '156', 'white'), ('dataset/JPEGImages/00000.jpg', '429', '98', '474', '163', 'white'), ('dataset/JPEGImages/00000.jpg', '503', '100', '552', '166', 'white'), ('dataset/JPEGImages/00000.jpg', '588', '115', '637', '177', 'white'), ('dataset/JPEGImages/00000.jpg', '694', '92', '743', '164', 'white'), ('dataset/JPEGImages/00000.jpg', '763', '57', '821', '129', 'white'), ('dataset/JPEGImages/00000.jpg', '827', '71', '869', '129', 'white'), ('dataset/JPEGImages/00000.jpg', '860', '89', '915', '155', 'white'), ('dataset/JPEGImages/00000.jpg', '922', '67', '1001', '155', 'white')]


In [115]:
def splitAnnotations(annotations:dict, ratio:float = 0.7) -> Tuple[dict, dict]:
    limit = int(len(annotations) * ratio)
    train_dict = {}
    test_dict = {}
    for id in annotations:
        if limit > 0:
            train_dict[id] = annotations[id]
            limit -= 1
        else:
            test_dict[id] = annotations[id]
    return train_dict, test_dict
def saveCsv(filename:str, data:dict) -> None:
    with open(f'{filename}.csv', 'w', 'utf-8-sig') as f:
        csv_out = csv.writer(f)
        csv_out.writerow(['file_path','x_min', 'y_min', 'x_max', 'y_max', 'class'])
        for id in data:
            for row in data[id]:
                csv_out.writerow(row)
def saveTxt(filename:str, data:dict) -> None:
    with open(f'{filename}.txt', 'w', 'utf-8-sig') as f:
        for id in data:
            for line in data[id]:
                f.write(','.join(line))
                f.write('\n')

In [116]:
train, test = splitAnnotations(annotations, ratio=0.7)
saveCsv('train_data', train)
saveCsv('test_data', test)
saveTxt('train_data', train)
saveTxt('test_data', test)